# CQW - Small Reports
Updated: 2019-10-16

Small reports typically have run times that are less than a few seconds. The following reports are found in this notebook. 

  - Report 1c1  - Store Sales for the Year 2000
  - Report 1c2  - Store Sales for the Year 2000 by Quarter
  - Report 1d1  - Category, Class, and Brand Sales
  - Report 1d2  - Sales by Stores in Alabama
  - Report 1e1  - Top 3 Item Brands by SubTotal Revenue
  - Report 1e1b - Top 3 Item Brands by Wholesale Cost
  - Report 1e1c - Top 3 Item Brands by Net Profits
  - Report 1e2  - Item Brand Sales by Store


### Load Db2 Extensions and Connect to the Database
The `connection` notebook contains the `CONNECT` statement which allows access to the `SAMPLE` database. If you need to modify the connection information, edit the `connection.ipynb` notebook.

In [ ]:
%run ../db2.ipynb
%run ../connection.ipynb

### Create a Function for Running the Queries
This function is used to time the queries and return the total time taken. The parameter is the SQL that will be run and the function will execute against the row-based and col-based tables. 

In [ ]:
def runquery(sql):
    
    %sql -q SET CURRENT SCHEMA = CQWROW
        
    print("Starting execution of ROW query")    
    start_time = time.time()

    %sql -q {sql}
    
    end_time = time.time()
    row = end_time - start_time
    
    print("Row run time = {0}".format(row))
    
    %sql -q SET CURRENT SCHEMA = CQWCOL
    
    print()
    print("Starting execution of COLUMNAR query")        
    
    start_time = time.time()

    results = %sql -q {query}
    
    end_time = time.time()
    col = end_time - start_time
    
    print("Column run time = {0}".format(col))    
    
    pdisplay(results)
    
    %sql -bar VALUES ('Row',:row),('Column',:col)

## Store Sales for the Year 2000

In [ ]:
query = '''
SELECT
    "DATE_DIM"."D_YEAR" AS "Date_D_YEAR", 
    SUM(CAST("STORE_SALES"."SS_QUANTITY" AS DECIMAL(31,2))) AS "Quantity__Store_Sales_"
FROM
    "DATE_DIM" "DATE_DIM"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "DATE_DIM"."D_DATE_SK" = "STORE_SALES"."SS_SOLD_DATE_SK" 
WHERE 
    "DATE_DIM"."D_YEAR" IN ( 
        2000 ) 
GROUP BY 
    "DATE_DIM"."D_YEAR"
'''
runquery(query)

## Store Sales for the Year 2000 by Quarter

In [ ]:
query = '''
SELECT
    "DATE_DIM"."D_YEAR" AS "Date_D_YEAR", 
    "DATE_DIM"."D_QOY" AS "Date_D_QOY", 
    SUM(CAST("STORE_SALES"."SS_QUANTITY" AS DECIMAL(31,2))) AS "Quantity__Store_Sales_"
FROM
    "DATE_DIM" "DATE_DIM"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "DATE_DIM"."D_DATE_SK" = "STORE_SALES"."SS_SOLD_DATE_SK" 
WHERE 
    "DATE_DIM"."D_YEAR" IN ( 
        2000 ) 
GROUP BY 
    "DATE_DIM"."D_YEAR", 
    "DATE_DIM"."D_QOY"
'''
runquery(query)

## Category, Class, and Brand Sales

In [ ]:
query = '''
SELECT
    "ITEM"."I_CATEGORY" AS "Items_Trimmed_Category", 
    "ITEM"."I_CLASS" AS "Items_Trimmed_Clasification", 
    "ITEM"."I_BRAND" AS "Items_Trimmed_Brand", 
    SUM("STORE_SALES"."SS_NET_PAID") * SUM(CAST("STORE_SALES"."SS_QUANTITY" AS DECIMAL(31,2))) AS "SubTotal_Revenue__Store_Sales_"
FROM
    "ITEM" "ITEM"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "ITEM"."I_ITEM_SK" = "STORE_SALES"."SS_ITEM_SK" 
GROUP BY 
    "ITEM"."I_CATEGORY", 
    "ITEM"."I_CLASS", 
    "ITEM"."I_BRAND"
'''
runquery(query)

## Sales by Stores in Tennesee

In [ ]:
query = '''
SELECT
    "STORE"."S_STORE_SK" AS "Stores_S_STORE_SK", 
    (SUM("STORE_SALES"."SS_NET_PAID") - SUM(CAST("STORE_SALES"."SS_WHOLESALE_COST" AS DECIMAL(31,2)))) * SUM(CAST("STORE_SALES"."SS_QUANTITY" AS DECIMAL(31,2)))
 AS "SubTotal_Gross_Profit__Store_Sales_"
FROM
    "STORE" "STORE"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "STORE"."S_STORE_SK" = "STORE_SALES"."SS_STORE_SK" 
WHERE 
    "STORE"."S_COUNTRY" IN ( 
        'United States' ) AND
    "STORE"."S_STATE" IN ( 
        'TN' ) 
GROUP BY 
    "STORE"."S_STORE_SK"
'''
runquery(query)

## Top 3 Item Brands by SubTotal Revenue

In [ ]:
query = '''
SELECT
    "ITEM"."I_CATEGORY" AS "Items_Trimmed_Category", 
    "ITEM"."I_CLASS" AS "Items_Trimmed_Clasification", 
    "ITEM"."I_BRAND" AS "Items_Trimmed_Brand", 
    AVG(CAST("STORE_SALES"."SS_SALES_PRICE" AS DOUBLE PRECISION)) AS "Sales_Price__Store_Sales_",
    SUM(CAST("STORE_SALES"."SS_QUANTITY" AS DECIMAL(31,2))) AS "Quantity__Store_Sales_"
FROM
    "ITEM" "ITEM"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "ITEM"."I_ITEM_SK" = "STORE_SALES"."SS_ITEM_SK" 
WHERE 
    "ITEM"."I_CATEGORY" IN ( 
        'Music', 
        'Men', 
        'Children' ) AND
    "ITEM"."I_CLASS" IN ( 
        'accessories', 
        'classical', 
        'infants' ) AND
    "ITEM"."I_BRAND" IN ( 
        'importoexporti #2', 
        'amalgimporto #2', 
        'edu packscholar #2' ) 
GROUP BY 
    "ITEM"."I_CATEGORY", 
    "ITEM"."I_CLASS", 
    "ITEM"."I_BRAND"
'''
runquery(query)

##  Top 3 Item Brands by Wholesale Cost

In [ ]:
query = '''
SELECT
    "ITEM"."I_CATEGORY" AS "Items_Trimmed_Category", 
    "ITEM"."I_CLASS" AS "Items_Trimmed_Clasification", 
    "ITEM"."I_BRAND" AS "Items_Trimmed_Brand",
    AVG(CAST("STORE_SALES"."SS_LIST_PRICE" AS DOUBLE PRECISION)) AS "List_Price__Store_Sales_",
    SUM(CAST("STORE_SALES"."SS_WHOLESALE_COST" AS DECIMAL(31,2))) AS "Wholesale__Cost__"
FROM
    "ITEM" "ITEM"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "ITEM"."I_ITEM_SK" = "STORE_SALES"."SS_ITEM_SK" 
WHERE 
    "ITEM"."I_CATEGORY" IN ( 
        'Music', 
        'Men', 
        'Children' ) AND
    "ITEM"."I_CLASS" IN ( 
        'accessories', 
        'classical', 
        'infants' )  AND
    "ITEM"."I_BRAND" IN (
        'importoexporti #2', 
        'amalgimporto #2', 
        'edu packscholar #2' )
GROUP BY 
    "ITEM"."I_CATEGORY", 
    "ITEM"."I_CLASS",
    "ITEM"."I_BRAND"
'''
runquery(query)

## Top 3 Item Brands by Net Profits

In [ ]:
query = '''
SELECT
    "ITEM"."I_CATEGORY" AS "Items_Trimmed_Category", 
    "ITEM"."I_CLASS" AS "Items_Trimmed_Clasification", 
    "ITEM"."I_BRAND" AS "Items_Trimmed_Brand", 
    SUM("STORE_SALES"."SS_NET_PROFIT") AS "Net_Profit__Stor",
    SUM("STORE_SALES"."SS_COUPON_AMT") AS "Coupon_Amount__S",
    SUM("STORE_SALES"."SS_NET_PAID" - CAST("STORE_SALES"."SS_WHOLESALE_COST" AS DECIMAL(31,2))) AS "Gross_Profit__St"
FROM
    "ITEM" "ITEM"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "ITEM"."I_ITEM_SK" = "STORE_SALES"."SS_ITEM_SK" 
WHERE 
    "ITEM"."I_CATEGORY" IN ( 
        'Music', 
        'Men', 
        'Children' ) AND
    "ITEM"."I_CLASS" IN ( 
        'accessories', 
        'classical', 
        'infants' ) AND
    "ITEM"."I_BRAND" IN ( 
        'importoexporti #2', 
        'amalgimporto #2', 
        'edu packscholar #2' ) 
GROUP BY 
    "ITEM"."I_CATEGORY", 
    "ITEM"."I_CLASS", 
    "ITEM"."I_BRAND"
'''
runquery(query)

##  Item Brand Sales by Store

In [ ]:
query = '''
SELECT
    "ITEM"."I_CATEGORY" AS "Items_Trimmed_Category", 
    "ITEM"."I_CLASS" AS "Items_Trimmed_Clasification", 
    "ITEM"."I_BRAND" AS "Items_Trimmed_Brand", 
    SUM("STORE_SALES"."SS_NET_PAID") * SUM(CAST("STORE_SALES"."SS_QUANTITY" AS DECIMAL(31,2))) AS "SubTotal_Revenue__Store_Sales_"
FROM
    "ITEM" "ITEM"
        INNER JOIN "STORE_SALES" "STORE_SALES"
        ON "ITEM"."I_ITEM_SK" = "STORE_SALES"."SS_ITEM_SK" 
GROUP BY 
    "ITEM"."I_CATEGORY", 
    "ITEM"."I_CLASS", 
    "ITEM"."I_BRAND"
'''
runquery(query)

#### Credits: IBM 2019, George Baklarz [baklarz@ca.ibm.com]